# Leakage Detection Framework
This is a framework for TVLA. The goal of this framework is to make it easier for researchers, scientists, or anyone interested in using TVLA for leakage detection.

For basic functionality, the only class that needs to be imported is the *LeakageDetectionFramework*.

In [1]:
from core import LeakageDetectionFramework
import numpy as np

Using an instance of the *LeakageDetectionFramework*, we can already do a few basic things. We can already load trace data from an *npz* file that will automatically be loaded into and interpreted by a data handler class. We will touch on this later. For now we will focus on the framework instance itself, because the rest is happening under the hood.
The following code will load the trace data from the *npz* file and using Welch's t-test, it will calculate the t statistic.

In [3]:
# Create the instance
ldf = LeakageDetectionFramework()

# Load the data
ldf.load_data("../traces/REASSURE_power_Unprotected_AES_fixed_vs_random_Exp1.npz")

# Calculate t_statistic using Welch's t-test
t_statistic = ldf.calculate_t_statistic()

# Get the indices of the leaky samples for further analysis
leakage = ldf.indices_of_leaky_samples(t_statistic, range(0,13900), 4.5)

print('Results for t-satistics: %s' % t_statistic)

# We can print, for example the number of total leaky points
leaky_indices = leakage==True
print(leakage[leaky_indices].sum())


Results for t-satistics: [-0.22990521  0.49794166  0.65197626 ... 33.54417795 32.84687168
 32.03226545]
9390


In [3]:
TraceData.get_instance().get_all_traces()

NameError: name 'TraceData' is not defined